In [129]:
import dalex as dx
import numpy as np
import pandas as pd

In [130]:
dx.__version__

'1.5.0'

In [131]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

data = pd.read_csv('ai4i2020.csv')
data.columns =  data.columns.str.replace(' ','_')
data.columns =  data.columns.str.replace(r'\[','', regex=True)
data.columns =  data.columns.str.replace(r'\]','', regex=True)
data.head()

,UDI,Product_ID,Type,Air_temperature_K,Process_temperature_K,Rotational_speed_rpm,Torque_Nm,Tool_wear_min,Machine_failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [132]:
processed_data = data.drop(['UDI', 'Product_ID'], axis=1).copy()
processed_data.columns


Index(['Type', 'Air_temperature_K', 'Process_temperature_K',
       'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 'Machine_failure',
       'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [133]:
X = processed_data.drop(['Machine_failure'], axis=1).copy()
y = processed_data['Machine_failure']

In [134]:
categorical_features = ['Type']
numerical_features = [ 'Air_temperature_K', 'Process_temperature_K',
       'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_features),
        ('num', 'passthrough', numerical_features)])

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(learning_rate= 0.1, max_depth= 3, n_estimators= 140))])

clf.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Type']),
                                                 ('num', 'passthrough',
                                                  ['Air_temperature_K',
                                                   'Process_temperature_K',
                                                   'Rotational_speed_rpm',
                                                   'Torque_Nm', 'Tool_wear_min',
                                                   'TWF', 'HDF', 'PWF', 'OSF',
                                                   'RNF'])])),
                ('classifier',
                 XGBClassifier(base_score=0.5, booste...
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.1, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=3, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=140, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [135]:
exp = dx.Explainer(clf, X, y)

Preparation of a new explainer is initiated

  -> data              : 10000 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 10000 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x00000284954B5090> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 8.55e-05, mean = 0.0339, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.0571, mean = 2.05e-05, max = 0.985
  -> model_info        : package sklearn

A new explainer has been created!


In [136]:
import plotly
exp.model_performance().result

,recall,precision,f1,accuracy,auc
XGBClassifier,0.973451,1.0,0.986547,0.9991,0.999975


In [137]:
failures = data.loc[data['Machine_failure'] >  0]
failures['Type'].value_counts()
protected = data['Type']
privileged  = 'L' # since most failures are from L type

In [138]:
fobject = exp.model_fairness(protected = protected, privileged=privileged)

In [139]:
fobject.fairness_check(epsilon = 0.8)

Bias detected in 1 metric: STP

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on 'L'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR  ACC  PPV  FPR       STP
H  0.968464  1.0  1.0  NaN  0.526316
M  0.968464  1.0  1.0  NaN  0.684211

Warning!
Take into consideration that NaN's are present, consider checking 'metric_scores' plot to see the difference


In [140]:
fobject.result

,TPR,TNR,PPV,NPV,FNR,FPR,FDR,FOR,ACC,STP
H,0.968464,1.0,1.0,1.0,2.823529,NaN,NaN,1.0,1.0,0.526316
L,1.000000,1.0,1.0,1.0,1.000000,NaN,NaN,1.0,1.0,1.000000
M,0.968464,1.0,1.0,1.0,2.823529,NaN,NaN,1.0,1.0,0.684211


In [141]:
fobject.plot()


Found NaN's or 0's for models: {'XGBClassifier'}
It is advisable to check 'metric_ratios'


In [142]:
fobject.plot(type = 'metric_scores')

In [143]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [144]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numerical_features)])

clf_forest = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=123, max_depth=5))]).fit(X,y)

clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(random_state=123))]).fit(X,y)

In [145]:
# create Explainer objects 
exp_forest  = dx.Explainer(clf_forest, X,y, verbose = False)
exp_logreg  = dx.Explainer(clf_logreg, X,y, verbose = False)
# create fairness explanations
fobject_forest = exp_forest.model_fairness(protected, privileged)
fobject_logreg = exp_logreg.model_fairness(protected, privileged)

In [146]:
fobject.plot(objects=[fobject_forest, fobject_logreg])


Found NaN's or 0's for models: {'LogisticRegression', 'RandomForestClassifier', 'XGBClassifier'}
It is advisable to check 'metric_ratios'


In [147]:
fobject.plot(objects=[fobject_forest, fobject_logreg], type = "metric_scores")

In [148]:
fobject.parity_loss

TPR    0.064088
TNR    0.000000
PPV    0.000000
NPV    0.000000
FNR    2.075975
FPR         NaN
FDR         NaN
FOR    0.000000
ACC    0.000000
STP    1.021344
dtype: float64

In [149]:
fobject.plot(objects=[fobject_forest, fobject_logreg], type = "radar")

Found NaNs in following models: {'LogisticRegression', 'RandomForestClassifier', 'XGBClassifier'}


In [150]:
fobject.plot(objects=[fobject_forest, fobject_logreg], type = "heatmap")

Found NaNs in following models: {'LogisticRegression', 'RandomForestClassifier', 'XGBClassifier'}


In [151]:
fobject.plot(objects=[fobject_forest, fobject_logreg], type = "performance_and_fairness")